In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import cv2
import random
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data.dataset import random_split

In [2]:
def print_params(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.data)

In [3]:
def layers_connexion(model,input_size):
    Layers = 1 #input layer
    Connexions = []
    Nodes = [input_size]
    
    for name, param in model.named_parameters():
        if param.requires_grad:
            if "bias" in name:
                Layers += 1
                Nodes.append(len(param.detach().numpy()))
            else:
                a = param.detach().numpy()
            #Replace weight value by binary number
                a = np.select( [a != 0.0, a== 0.0], [1,0],default = a)
                a = np.array(a)
                Connexions.append(a)
    return(Layers, Connexions, Nodes)

In [5]:
class Net_Task2(torch.nn.Module):
    def __init__(self,input_dimension, output_dimension):
        super(Net_Task2, self).__init__()
        self.fc1 = nn.Linear(in_features = input_dimension, out_features = 5)
        self.fc2 = nn.Linear(in_features = 5, out_features = 5)
        self.fc3 = nn.Linear(in_features = 5, out_features = 5)
        #self.fc4 = nn.Linear(in_features = 5, out_features = 5)
        #self.fc5 = nn.Linear(in_features = 5,  out_features = output_dimension)
        self.fc4 = nn.Linear(in_features = 5,  out_features = output_dimension)

        #self.mask2 = torch.tensor([[1,0],[1,0],[0,1],[0,1]])
        #self.mask3 = torch.tensor([[1,1,0,0],[0,0,1,1]]) 


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        #x = self.fc5(x)
        x = self.fc4(x)
        return torch.sigmoid(x)

class Net_Task1(torch.nn.Module):
    def __init__(self,input_dimension, output_dimension):
        super(Net_Task1, self).__init__()
        self.fc1 = nn.Linear(in_features = input_dimension, out_features = 1)
        self.fc2 = nn.Linear(in_features = 1, out_features = 2)
        self.fc3 = nn.Linear(in_features = 2,  out_features = output_dimension)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.sigmoid(x)



In [6]:
model = Net_Task2(input_dimension = 5, output_dimension = 1)
print('\n The model parameters before the update are: \n')
print_params(model)


 The model parameters before the update are: 

fc1.weight tensor([[ 0.3438, -0.3645, -0.2179,  0.1977,  0.2077],
        [ 0.1628, -0.1337, -0.1047,  0.0086,  0.2361],
        [-0.0203,  0.4032, -0.1586, -0.1919,  0.2630],
        [-0.4333, -0.0190,  0.0519,  0.1863, -0.2896],
        [ 0.0324,  0.0712,  0.3107, -0.1679, -0.3326]])
fc1.bias tensor([ 0.1212, -0.2219,  0.4222,  0.3586, -0.3397])
fc2.weight tensor([[-0.3704,  0.2508, -0.0796, -0.0917, -0.2026],
        [-0.1882, -0.4452, -0.2178,  0.1285,  0.4225],
        [ 0.3439, -0.3233,  0.0090, -0.2158, -0.1013],
        [ 0.3259,  0.3778, -0.1262, -0.3755, -0.1559],
        [-0.2813, -0.3646, -0.0117, -0.3899,  0.1150]])
fc2.bias tensor([ 0.2860,  0.0238, -0.3747,  0.1253,  0.3516])
fc3.weight tensor([[-0.1678, -0.3700,  0.1195, -0.1478,  0.1883],
        [-0.1499,  0.2549,  0.2908, -0.0652,  0.0855],
        [-0.0502, -0.1304,  0.1478, -0.3118,  0.0192],
        [ 0.1198,  0.0637, -0.2314,  0.3651, -0.3822],
        [-0.1498, -0.

KeyboardInterrupt: 

In [14]:
class Network_Visualization:
    def __init__(self,display,magnification,Model,input_size):
        self.layers,self.connexions,self.nodes = layers_connexion(model,input_size)
        # Set whether the environment should be displayed after every step
        self.display = display
        # Set the magnification factor of the display
        self.magnification = magnification
        # Create an image which will be used to display the environment
        self.image = np.zeros([int(self.magnification), int(self.magnification), 3], dtype=np.uint8)
        # Set the width and height of the environment
        self.width = 5
        self.height = 5
        self.rayon = 20
            
        # Init y coordinates of each layers
        self.b = np.linspace(4* self.rayon,
                        self.magnification-4*self.rayon,
                        self.layers)
        # Init coordinates of each nodes, i//self.layers allow to have the same y coordinates for every nodes of the same layer
        self.Nodes_coordinates = []
        
    def draw(self):
        # Create the background image
        window_top_left = (0, 0)
        window_bottom_right = (self.magnification , self.magnification )
        cv2.rectangle(self.image, window_top_left, window_bottom_right, (255, 255, 255), thickness=cv2.FILLED)
        for i in range(self.layers):
            a = np.linspace(4* self.rayon,
                            self.magnification-4*self.rayon,
                            self.nodes[i])
            coordinates = [[int(a[j]),int(self.b[i])] for j in range(len(a))]
            self.Nodes_coordinates.append(coordinates)     
            
        #Draw the Nodes
        
        for i in range(self.layers):
            for j in range(len(self.Nodes_coordinates[i])):
                cv2.circle(self.image,center= self.Nodes_coordinates[i][j], 
                           radius = self.rayon, 
                           color =(139,139,0), 
                           thickness = 2 )
        #Draw the connexions
        for i in reversed(range(1, self.layers )):
            g = self.connexions[i-1].shape
            for j in range(g[0]):
                for k in range(g[1]):
                    if self.connexions[i-1][j][k] != 0:
                      
                        cv2.line(self.image, 
                                 pt1 = np.array(self.Nodes_coordinates[i-1][k]) + [0,self.rayon], 
                                 pt2 = np.array(self.Nodes_coordinates[i][j]) - [0,self.rayon],
                                 color =  (238,229,142) , 
                                 thickness = 2)

        cv2.imshow("Neural Network", self.image)
        # This line is necessary to give time for the image to be rendered on the screen
        cv2.waitKey(1)
        

In [15]:
display_on = True
visu = Network_Visualization(display = display_on,magnification=800,Model = model,input_size=5)

# Determine the time at which training will stop, i.e. in 10 minutes (600 seconds) time
start_time = time.time() 
end_time = start_time + 30
visu.draw()
#while time.time() < end_time:
# Optionally, show the environment
 #   if display_on:
  #      visu.draw()

In [9]:
h = [[0, 6720.0], [0, 6720.0], [0, 6720.0], [0, 6720.0], [0, 6720.0], [0, 6560.0], [0, 6560.0], [0, 6560.0], [0, 6560.0], [0, 6560.0], [0, 6400.0], [0, 6400.0], [0, 6400.0], [0, 6400.0], [0, 6400.0], [0, 6240.0], [0, 6240.0], [0, 6240.0], [0, 6240.0], [0, 6240.0], [0, 6080.0]]

In [ ]:
print(h[:])

In [ ]:
      z_ = np.select( (z_==1,z_==2, z_==3),
                (150, 120, 110),
                default=z_)

In [26]:

magnification = 800
height = 5
width = 5
image = np.zeros([int(magnification), int(magnification), 3], dtype=np.uint8)
window_top_left = (0, 0)
window_bottom_right = (magnification*height, magnification*width)
cv2.rectangle(image, window_top_left, window_bottom_right, (0, 238, 229), thickness=cv2.FILLED)
cv2.imshow("Environment6",image)


In [43]:
print(3<4)

True
